In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
import os
os.chdir('/content/drive/My Drive/translate')

Mounted at /content/drive


In [ ]:
!pip install sacrebleu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.9/118.9 kB 2.2 MB/s eta 0:00:00


In [ ]:
import tensorflow as tf
import numpy as np
import json
from sacrebleu import BLEU
import pickle

In [ ]:
def load_text_vectorization(path):
    from_disk = pickle.load(open(path, "rb"))
    tv = tf.keras.layers.TextVectorization.from_config(from_disk['config'])
    tv.set_weights(from_disk['weights'])
    return tv

In [ ]:
bleu = BLEU()
import os
os.environ['TF_GPU_ALLOCATOR'] = 'cuda_malloc_async'

model_name = 'LSTM_PTT_2023_08_06_VS20000_SL20_Fri_Aug_11_185708_2023.keras'
split_char = '⫯'
max_decoded_sentence_length = 20

tv_name = 'PTT_2023_08_06'
source_vectorization = load_text_vectorization(f"models/{tv_name}_source_vectorization.pkl")
target_vectorization = load_text_vectorization(f"models/{tv_name}_target_vectorization.pkl")

In [ ]:
zh_vocab = target_vectorization.get_vocabulary()
zh_index_lookup = dict(zip(range(len(zh_vocab)), zh_vocab))
model = tf.keras.models.load_model(f'models/{model_name}')

In [ ]:
with open(f'datasets/{tv_name}_engTyping_inserted_lines.txt', encoding='utf8') as file:
    engTyping_inserted_lines = file.readlines()
with open(f'datasets/{tv_name}_zh_lines.txt', encoding='utf8') as file:
    zh_lines = file.readlines()
lines_len = len(engTyping_inserted_lines)
assert lines_len == len(zh_lines)
engTyping_inserted_lines = engTyping_inserted_lines[int(lines_len * 0.85):]
zh_lines = zh_lines[int(lines_len * 0.85):]
for i in range(len(zh_lines)):
    zh_lines[i] = ' '.join(zh_lines[i].split(split_char)[1:-1])

In [ ]:
pred_sentences = []
loop_times = 200
eng_len = len(engTyping_inserted_lines)
split_point = eng_len // loop_times
for k in range(loop_times + 1):
    lines = engTyping_inserted_lines[k * split_point:(k + 1) * split_point]
    len_lines = len(lines)
    tokenized_input_sentence = source_vectorization(lines)
    decoded_sentences = ["[start]"] * len_lines
    for i in range(max_decoded_sentence_length):
        tokenized_target_sentence = target_vectorization(
            decoded_sentences)[:, :-1]
        predictions = model(
            [tokenized_input_sentence, tokenized_target_sentence])
        for j in range(len_lines):
            if decoded_sentences[j].endswith('[end]'): continue
            decoded_sentences[j] += split_char + zh_index_lookup[np.argmax(predictions[j, i, :])]
    pred_sentences.extend(decoded_sentences)
    print(f'\r{k}/{loop_times}', end='')


print(pred_sentences)

199/200

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [ ]:
count = 0
for i in range(len(pred_sentences)):
    pred_sentences[i] = pred_sentences[i][7:]
    if pred_sentences[i].endswith('[end]'): pred_sentences[i] = pred_sentences[i][:-5]
    pred_sentences[i] = pred_sentences[i].replace(split_char, ' ')
    count += 1
    print(f'{count}/{eng_len}', end='\r')

result = bleu.corpus_score(pred_sentences, [zh_lines])
print(result)

BLEU = 78.56 86.6/81.8/77.3/73.0 (BP = 0.988 ratio = 0.988 hyp_len = 2874646 ref_len = 2909826)


In [ ]:
len(pred_sentences)


231813

In [ ]:
len(zh_lines)


231813